In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import lightgbm as lgb

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
import optuna

from common import EP
from dfdb import DFDB

import types
import copy

Using TensorFlow backend.


In [2]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [3]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_colwidth', -1)

In [4]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [5]:
df_spec_train = pd.read_pickle('../feats/spec_features.pkl')
df_spec_test = pd.read_pickle('../feats/spec_features_test.pkl')

In [6]:
df_train = pd.merge(df_train, df_spec_train, on='index')
df_test = pd.merge(df_test, df_spec_test, on='index')

In [7]:
test_X = df_train.drop(columns=['y','index','group','label']).copy()
test_X.index = df_train['index']
test_y = df_train['y'].copy()
test_y.index = df_train['index']
tsfresh_columns = select_features(test_X, test_y).columns.tolist()

In [8]:
db = DFDB('../trial/lasso.pkl', auto_commit=False)

In [9]:
df_trial = db.select()
df_trial['kfold-type'] = df_trial['param'].apply(lambda x: x['kfold']['type'])
df_trial['algorithm-init'] = df_trial['param'].apply(lambda x: x['algorithm']['init'])

In [10]:
df_trial[(df_trial['kfold-type']=='group')&(df_trial['mae_diff']<.1)][['datetime','nfeatures', 'remark', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark','algorithm-init']].sort_values(by=['val_mae'], ascending=True).head()

,datetime,nfeatures,remark,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark,algorithm-init
341,2019-05-25 05:40:15.130760,20,tune 116,2.101109,0.003703,2.124651,0.015621,0.023541,tune 116,"{'alpha': 0.003240282554718643, 'fit_intercept': True, 'normalize': False, 'precompute': False, 'copy_X': True, 'max_iter': 2440, 'tol': 0.0002829095714346777, 'warm_start': False, 'positive': False, 'random_state': 195, 'selection': 'cyclic'}"
291,2019-05-25 05:39:57.821381,20,tune 116,2.112033,0.003890,2.129703,0.016012,0.017670,tune 116,"{'alpha': 0.01589296450521438, 'fit_intercept': True, 'normalize': False, 'precompute': False, 'copy_X': True, 'max_iter': 3160, 'tol': 0.00030733041227957097, 'warm_start': False, 'positive': False, 'random_state': 3826, 'selection': 'cyclic'}"
117,2019-05-22 13:54:00.119986,42,start from top1 column,2.123888,0.003288,2.132280,0.013294,0.008391,start from top1 column,"{'alpha': 0.1, 'fit_intercept': True, 'normalize': False, 'precompute': False, 'copy_X': True, 'max_iter': 1000, 'tol': 0.0001, 'warm_start': False, 'positive': False, 'random_state': 42, 'selection': 'cyclic'}"
116,2019-05-22 13:53:59.563911,41,start from top1 column,2.123888,0.003288,2.132280,0.013294,0.008391,start from top1 column,"{'alpha': 0.1, 'fit_intercept': True, 'normalize': False, 'precompute': False, 'copy_X': True, 'max_iter': 1000, 'tol': 0.0001, 'warm_start': False, 'positive': False, 'random_state': 42, 'selection': 'cyclic'}"
115,2019-05-22 13:53:59.022892,41,start from top1 column,2.124411,0.003284,2.132540,0.013255,0.008129,start from top1 column,"{'alpha': 0.1, 'fit_intercept': True, 'normalize': False, 'precompute': False, 'copy_X': True, 'max_iter': 1000, 'tol': 0.0001, 'warm_start': False, 'positive': False, 'random_state': 42, 'selection': 'cyclic'}"


In [11]:
catboost_columns = ['mfcc_3_rolling_std_mean',
  'max_9',
  'q25_roll_std_100',
  'max_to_min',
  'max_to_min_5',
  'iqr_6',
  'q05_roll_std_1000',
  'q05_roll_std_100',
  'abs_max_4',
  'abs_max_1',
  'spkt_welch_densitycoeff_2',
  'spkt_welch_density__coeff_3',
  'abs_q75_7',
  'q01_2',
  'abs_q01_4',
  'max_to_min_diff_5',
  'q05_5',
  'abs_q25_5',
  'abs_max_2',
  'min__roll_std',
  'median__roll_std',
  'abs_max_7',
  '5000peak_peak_amp_max_',
  'abs_q75_6']
lgbm_columns = ['mfcc_10_abs_q75',
  'q25_roll_std_100',
  'iqr_6',
  'mfcc_9_mean',
  'abs_q75_7',
  'mfcc_delta_5_min',
  'mfcc_delta_3_quantile01',
  'abs_q75_6',
  'q05_roll_std_100',
  'mfcc_accelerate_1_kurtosis',
  'mfcc_5_mean',
  'spkt_welch_density__coeff_42',
  'mfcc_3_rolling_std_mean',
  'mfcc_12_mean',
  '5000smoothness_entropy_',
  'mfcc_13_mean',
  "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
  'mfcc_5_abs_max',
  'abs_q25_5',
  'mfcc_5_quantile95',
  'spkt_welch_densitycoeff_5',
  '5000crest_factor_quantile75',
  'spkt_welch_densitycoeff_2',
  'mfcc_accelerate_8_variance',
  "number_peaks{'n': 10}",
  'spkt_welch_density__coeff_3']
xgbm_columns = ['q05_roll_std_1000',
  'q05_roll_std_100',
  'mfcc_9_mean',
  'abs_q01_4',
  'iqr_6',
  'spkt_welch_density__coeff_4',
  'abs_q25_5',
  'abs_q75_6',
  'q01_2',
  "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
  'abs_max_4',
  'median__roll_std',
  'abs_max_7',
  'spkt_welch_density__coeff_28',
  '5000crest_factor_quantile75',
  'mfcc_1_kurtosis',
  'mfcc_4_median',
  'q05_5',
  'abs_max_8',
  'abs_q75_7',
  'q25_roll_std_100',
  'mfcc_delta_3_quantile01',
  'spkt_welch_densitycoeff_2',
  'max_to_min_diff_5',
  'mfcc_12_mean',
  'spkt_welch_densitycoeff_5',
  'spkt_welch_density__coeff_3',
  "number_peaks{'n': 10}",
  'spkt_welch_density__coeff_27']

In [12]:
all_columns = catboost_columns+lgbm_columns+xgbm_columns
unique_columns = list(set(all_columns))
common_columns = []
common_columns50 = []
common_columns75 = []
common_columns95 = []
N_columns = 6
count_values = [all_columns.count(col) for col in unique_columns]
for col in unique_columns:
    if all_columns.count(col)==N_columns:
        common_columns.append(col)
    if all_columns.count(col)>=np.quantile(count_values, .5):
        common_columns50.append(col)
    if all_columns.count(col)>=np.quantile(count_values, .75):
        common_columns75.append(col)
    if all_columns.count(col)>=np.quantile(count_values, .95):
        common_columns95.append(col)
print('unique_columns ',len(unique_columns))
print('common_columns50 ',len(common_columns50))
print('common_columns75 ',len(common_columns75))
print('common_columns95 ',len(common_columns95))
print('common_columns ',len(common_columns))

unique_columns  47
common_columns50  24
common_columns75  24
common_columns95  8
common_columns  0


In [16]:
param = {'algorithm': {'cls': 'SVR',
  'fit': {},
  'init': {'kernel': 'rbf',
   'degree': 2,
   'gamma': 'auto',
   'coef0': 0.0,
   'tol': 0.004467335360842515,
   'C': 0.07777223472867957,
   'epsilon': 0.01,
   'shrinking': True}},
 'columns': unique_columns,
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler', 'init': {}}}

In [13]:
df_trial = db.select()

In [14]:
df_trial['kfold-type'] = df_trial['param'].apply(lambda x: x['kfold']['type'])

In [15]:
df_trial[(df_trial['kfold-type']=='group')].sort_values(by=['val_mae'])[['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].head()

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
341,2019-05-25 05:40:15.130760,tune 116,20,2.101109,0.003703,2.124651,0.015621,0.023541
291,2019-05-25 05:39:57.821381,tune 116,20,2.112033,0.003890,2.129703,0.016012,0.017670
117,2019-05-22 13:54:00.119986,start from top1 column,42,2.123888,0.003288,2.132280,0.013294,0.008391
116,2019-05-22 13:53:59.563911,start from top1 column,41,2.123888,0.003288,2.132280,0.013294,0.008391
115,2019-05-22 13:53:59.022892,start from top1 column,41,2.124411,0.003284,2.132540,0.013255,0.008129


In [19]:
param = {'algorithm': {'cls': 'Lasso',
  'fit': {},
  'init': {'alpha': 0.003240282554718643,
   'fit_intercept': True,
   'normalize': False,
   'precompute': False,
   'copy_X': True,
   'max_iter': 2440,
   'tol': 0.0002829095714346777,
   'warm_start': False,
   'positive': False,
   'random_state': 1985,
   'selection': 'cyclic'}},
 'columns': unique_columns,
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler', 'init': {}}}

In [20]:
mytrial = []
EP.width_frist_rfe(df_train, param, mytrial, 999, df_test=df_test, remark='wf new 341')

In [21]:
for trial_i in mytrial:
    db.insert(trial_i)
db.commit()

In [22]:
df_trial = db.select()
df_trial['kfold-type'] = df_trial['param'].apply(lambda x: x['kfold']['type'])
df_trial['algorithm-init'] = df_trial['param'].apply(lambda x: x['algorithm']['init'])

In [23]:
df_trial[(df_trial['remark']=='wf new 341')][['datetime','nfeatures', 'remark', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark','algorithm-init']].sort_values(by=['val_mae'], ascending=True).head()

,datetime,nfeatures,remark,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark,algorithm-init
2124,2019-06-02 03:13:13.797832,28,wf new 341,2.078826,0.003364,2.093322,0.014225,0.014496,wf new 341,"{'alpha': 0.003240282554718643, 'fit_intercept': True, 'normalize': False, 'precompute': False, 'copy_X': True, 'max_iter': 2440, 'tol': 0.0002829095714346777, 'warm_start': False, 'positive': False, 'random_state': 1985, 'selection': 'cyclic'}"
2070,2019-06-02 03:10:21.894488,29,wf new 341,2.078792,0.003369,2.093335,0.014227,0.014542,wf new 341,"{'alpha': 0.003240282554718643, 'fit_intercept': True, 'normalize': False, 'precompute': False, 'copy_X': True, 'max_iter': 2440, 'tol': 0.0002829095714346777, 'warm_start': False, 'positive': False, 'random_state': 1985, 'selection': 'cyclic'}"
2147,2019-06-02 03:14:24.140117,27,wf new 341,2.079083,0.003342,2.093356,0.014286,0.014273,wf new 341,"{'alpha': 0.003240282554718643, 'fit_intercept': True, 'normalize': False, 'precompute': False, 'copy_X': True, 'max_iter': 2440, 'tol': 0.0002829095714346777, 'warm_start': False, 'positive': False, 'random_state': 1985, 'selection': 'cyclic'}"
2143,2019-06-02 03:14:10.767921,27,wf new 341,2.079127,0.003398,2.093360,0.014239,0.014233,wf new 341,"{'alpha': 0.003240282554718643, 'fit_intercept': True, 'normalize': False, 'precompute': False, 'copy_X': True, 'max_iter': 2440, 'tol': 0.0002829095714346777, 'warm_start': False, 'positive': False, 'random_state': 1985, 'selection': 'cyclic'}"
2095,2019-06-02 03:11:45.159699,29,wf new 341,2.078758,0.003373,2.093362,0.014235,0.014604,wf new 341,"{'alpha': 0.003240282554718643, 'fit_intercept': True, 'normalize': False, 'precompute': False, 'copy_X': True, 'max_iter': 2440, 'tol': 0.0002829095714346777, 'warm_start': False, 'positive': False, 'random_state': 1985, 'selection': 'cyclic'}"


In [28]:
mytrial = []
#  tune hypterparameters
def objective(trial):

    alpha = trial.suggest_uniform('alpha', 0.001, 10)
    tol = trial.suggest_uniform('tol', 0.00001, 0.001)
    max_iter = trial.suggest_int('max_iter', 500, 5000)
    random_state = trial.suggest_int('random_state', 1, 9999)
    
        
    args={'algorithm': {'cls': 'Lasso',
      'fit': {},
      'init': {'alpha': alpha,
       'fit_intercept': True,
       'normalize': False,
       'precompute': False,
       'copy_X': True,
       'max_iter': max_iter,
       'tol': tol,
       'warm_start': False,
       'positive': False,
       'random_state': random_state,
       'selection': 'cyclic'}},
     'columns': df_trial.loc[2124]['param']['columns'],
     'kfold': {'n_splits': 3,
      'random_state': 1985,
      'shuffle': True,
      'type': 'stratified'},
     'scaler': {'cls': 'StandardScaler', 'init':{}}}
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 2124 by s3')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-06-02 05:32:07,918] Finished a trial resulted in value: 2.135256558294506. Current best value is 2.135256558294506 with parameters: {'alpha': 0.10747663327766874, 'tol': 0.00029260145984323795, 'max_iter': 4249, 'random_state': 6773}.
[I 2019-06-02 05:32:08,075] Finished a trial resulted in value: 3.0396783556986056. Current best value is 2.135256558294506 with parameters: {'alpha': 0.10747663327766874, 'tol': 0.00029260145984323795, 'max_iter': 4249, 'random_state': 6773}.
[I 2019-06-02 05:32:08,231] Finished a trial resulted in value: 3.0396783556986056. Current best value is 2.135256558294506 with parameters: {'alpha': 0.10747663327766874, 'tol': 0.00029260145984323795, 'max_iter': 4249, 'random_state': 6773}.
[I 2019-06-02 05:32:08,389] Finished a trial resulted in value: 3.0396783556986056. Current best value is 2.135256558294506 with parameters: {'alpha': 0.10747663327766874, 'tol': 0.00029260145984323795, 'max_iter': 4249, 'random_state': 6773}.
[I 2019-06-02 05:32:08,83

[I 2019-06-02 05:32:17,127] Finished a trial resulted in value: 3.0396783556986056. Current best value is 2.104192214813595 with parameters: {'alpha': 0.022581669532391782, 'tol': 0.0009242802662293123, 'max_iter': 4371, 'random_state': 7298}.
[I 2019-06-02 05:32:17,528] Finished a trial resulted in value: 2.185361089879335. Current best value is 2.104192214813595 with parameters: {'alpha': 0.022581669532391782, 'tol': 0.0009242802662293123, 'max_iter': 4371, 'random_state': 7298}.
[I 2019-06-02 05:32:17,759] Finished a trial resulted in value: 3.0396783556986056. Current best value is 2.104192214813595 with parameters: {'alpha': 0.022581669532391782, 'tol': 0.0009242802662293123, 'max_iter': 4371, 'random_state': 7298}.
[I 2019-06-02 05:32:17,954] Finished a trial resulted in value: 3.0396783556986056. Current best value is 2.104192214813595 with parameters: {'alpha': 0.022581669532391782, 'tol': 0.0009242802662293123, 'max_iter': 4371, 'random_state': 7298}.
[I 2019-06-02 05:32:18,15

[I 2019-06-02 05:32:27,626] Finished a trial resulted in value: 2.1477439438005117. Current best value is 2.098820784547915 with parameters: {'alpha': 0.017476305134883807, 'tol': 0.0008781373015129508, 'max_iter': 3790, 'random_state': 236}.
[I 2019-06-02 05:32:27,991] Finished a trial resulted in value: 2.360539894285673. Current best value is 2.098820784547915 with parameters: {'alpha': 0.017476305134883807, 'tol': 0.0008781373015129508, 'max_iter': 3790, 'random_state': 236}.
[I 2019-06-02 05:32:28,357] Finished a trial resulted in value: 2.296964912981841. Current best value is 2.098820784547915 with parameters: {'alpha': 0.017476305134883807, 'tol': 0.0008781373015129508, 'max_iter': 3790, 'random_state': 236}.
[I 2019-06-02 05:32:29,088] Finished a trial resulted in value: 2.1236381501284174. Current best value is 2.098820784547915 with parameters: {'alpha': 0.017476305134883807, 'tol': 0.0008781373015129508, 'max_iter': 3790, 'random_state': 236}.
[I 2019-06-02 05:32:29,435] Fi

[I 2019-06-02 05:32:39,917] Finished a trial resulted in value: 3.0396783556986056. Current best value is 2.092696552229216 with parameters: {'alpha': 0.012760245787000248, 'tol': 0.0006158799240305197, 'max_iter': 3373, 'random_state': 5471}.
[I 2019-06-02 05:32:40,379] Finished a trial resulted in value: 2.640384855544688. Current best value is 2.092696552229216 with parameters: {'alpha': 0.012760245787000248, 'tol': 0.0006158799240305197, 'max_iter': 3373, 'random_state': 5471}.
[I 2019-06-02 05:32:41,317] Finished a trial resulted in value: 2.104291114790899. Current best value is 2.092696552229216 with parameters: {'alpha': 0.012760245787000248, 'tol': 0.0006158799240305197, 'max_iter': 3373, 'random_state': 5471}.
[I 2019-06-02 05:32:41,537] Finished a trial resulted in value: 3.0396783556986056. Current best value is 2.092696552229216 with parameters: {'alpha': 0.012760245787000248, 'tol': 0.0006158799240305197, 'max_iter': 3373, 'random_state': 5471}.
[I 2019-06-02 05:32:41,959

[I 2019-06-02 05:32:53,159] Finished a trial resulted in value: 2.5242082181921677. Current best value is 2.0914964128707445 with parameters: {'alpha': 0.01175196743610787, 'tol': 0.0005884978218002106, 'max_iter': 2806, 'random_state': 8057}.
[I 2019-06-02 05:32:53,548] Finished a trial resulted in value: 2.684745913869991. Current best value is 2.0914964128707445 with parameters: {'alpha': 0.01175196743610787, 'tol': 0.0005884978218002106, 'max_iter': 2806, 'random_state': 8057}.
[I 2019-06-02 05:32:53,963] Finished a trial resulted in value: 2.333227173238381. Current best value is 2.0914964128707445 with parameters: {'alpha': 0.01175196743610787, 'tol': 0.0005884978218002106, 'max_iter': 2806, 'random_state': 8057}.
[I 2019-06-02 05:32:54,234] Finished a trial resulted in value: 3.0396783556986056. Current best value is 2.0914964128707445 with parameters: {'alpha': 0.01175196743610787, 'tol': 0.0005884978218002106, 'max_iter': 2806, 'random_state': 8057}.
[I 2019-06-02 05:32:54,483

[I 2019-06-02 05:33:11,907] Finished a trial resulted in value: 3.0396783556986056. Current best value is 2.084512486556774 with parameters: {'alpha': 0.004224535294107453, 'tol': 0.000314118227663159, 'max_iter': 2259, 'random_state': 9995}.
[I 2019-06-02 05:33:12,435] Finished a trial resulted in value: 2.4608716407291755. Current best value is 2.084512486556774 with parameters: {'alpha': 0.004224535294107453, 'tol': 0.000314118227663159, 'max_iter': 2259, 'random_state': 9995}.
[I 2019-06-02 05:33:12,708] Finished a trial resulted in value: 3.0396783556986056. Current best value is 2.084512486556774 with parameters: {'alpha': 0.004224535294107453, 'tol': 0.000314118227663159, 'max_iter': 2259, 'random_state': 9995}.
[I 2019-06-02 05:33:14,756] Finished a trial resulted in value: 2.092812883601996. Current best value is 2.084512486556774 with parameters: {'alpha': 0.004224535294107453, 'tol': 0.000314118227663159, 'max_iter': 2259, 'random_state': 9995}.
[I 2019-06-02 05:33:15,336] F

[I 2019-06-02 05:33:28,834] Finished a trial resulted in value: 2.4978762935177286. Current best value is 2.084512486556774 with parameters: {'alpha': 0.004224535294107453, 'tol': 0.000314118227663159, 'max_iter': 2259, 'random_state': 9995}.
[I 2019-06-02 05:33:29,109] Finished a trial resulted in value: 3.0396783556986056. Current best value is 2.084512486556774 with parameters: {'alpha': 0.004224535294107453, 'tol': 0.000314118227663159, 'max_iter': 2259, 'random_state': 9995}.


In [29]:
for trial_i in mytrial:
    db.insert(trial_i)
db.commit()

In [30]:
df_trial = db.select()
df_trial['kfold-type'] = df_trial['param'].apply(lambda x: x['kfold']['type'])
df_trial['algorithm-init'] = df_trial['param'].apply(lambda x: x['algorithm']['init'])

In [31]:
df_trial[(df_trial['remark']=='tune 2124 by s3')][['datetime','nfeatures', 'remark', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark','algorithm-init']].sort_values(by=['val_mae'], ascending=True).head()

,datetime,nfeatures,remark,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark,algorithm-init
2517,2019-06-02 05:33:11.155400,28,tune 2124 by s3,2.082355,0.000007,2.084512,0.000067,0.002157,tune 2124 by s3,"{'alpha': 0.004224535294107453, 'fit_intercept': True, 'normalize': False, 'precompute': False, 'copy_X': True, 'max_iter': 2259, 'tol': 0.000314118227663159, 'warm_start': False, 'positive': False, 'random_state': 9995, 'selection': 'cyclic'}"
2496,2019-06-02 05:32:58.223917,28,tune 2124 by s3,2.085342,0.000007,2.087458,0.000058,0.002116,tune 2124 by s3,"{'alpha': 0.007832271364984038, 'fit_intercept': True, 'normalize': False, 'precompute': False, 'copy_X': True, 'max_iter': 1566, 'tol': 0.0004554458972675173, 'warm_start': False, 'positive': False, 'random_state': 1292, 'selection': 'cyclic'}"
2533,2019-06-02 05:33:20.410760,28,tune 2124 by s3,2.086757,0.000008,2.088822,0.000053,0.002065,tune 2124 by s3,"{'alpha': 0.009289784770474098, 'fit_intercept': True, 'normalize': False, 'precompute': False, 'copy_X': True, 'max_iter': 1870, 'tol': 0.0004758495853608472, 'warm_start': False, 'positive': False, 'random_state': 9994, 'selection': 'cyclic'}"
2477,2019-06-02 05:32:49.882995,28,tune 2124 by s3,2.089534,0.000009,2.091496,0.000045,0.001963,tune 2124 by s3,"{'alpha': 0.01175196743610787, 'fit_intercept': True, 'normalize': False, 'precompute': False, 'copy_X': True, 'max_iter': 2806, 'tol': 0.0005884978218002106, 'warm_start': False, 'positive': False, 'random_state': 8057, 'selection': 'cyclic'}"
2504,2019-06-02 05:33:03.560592,28,tune 2124 by s3,2.089889,0.000009,2.091843,0.000044,0.001954,tune 2124 by s3,"{'alpha': 0.012062372547066533, 'fit_intercept': True, 'normalize': False, 'precompute': False, 'copy_X': True, 'max_iter': 1558, 'tol': 0.0005169385189011, 'warm_start': False, 'positive': False, 'random_state': 2175, 'selection': 'cyclic'}"
